In [2]:
%pip install matplotlib
%pip install seaborn
%matplotlib inline

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

Note: you may need to restart the kernel to use updated packages.



In [3]:
url = 'https://raw.githubusercontent.com/SamapanThongmee/SET_Index_Options_Premium_Analysis/main/SET50_Index_Futures.csv'
SET50_Index_Futures = pd.read_csv(url)
SET50_Index_Futures['Date']= pd.to_datetime(SET50_Index_Futures['Date'])
SET50_Index_Futures

,Date,Symbol,Open,High,Low,Close,SP,Vol,OI
0,2006-04-28,S50M06,536.5,542.0,535.6,536.2,536.0,152,98
1,2006-05-02,S50M06,536.0,537.2,536.0,536.1,536.1,53,114
2,2006-05-03,S50M06,537.0,538.5,537.0,538.5,538.5,75,132
3,2006-05-04,S50M06,536.8,536.8,535.6,535.7,535.7,81,166
4,2006-05-08,S50M06,536.5,542.0,536.5,541.0,541.0,109,165
...,...,...,...,...,...,...,...,...,...
16906,2023-11-24,S50Z23,868.0,872.0,863.0,864.0,864.0,178702,541068
16907,2023-11-27,S50Z23,862.0,864.0,856.0,861.0,861.0,199182,561082
16908,2023-11-28,S50Z23,864.0,870.0,862.0,868.0,868.0,186742,583118
16909,2023-11-29,S50Z23,871.0,872.0,858.0,858.0,858.0,238226,592627


In [5]:
indices = list(SET50_Index_Futures.groupby('Symbol').tail(1).index)
print(indices)

[39, 285, 532, 777, 1020, 1264, 1510, 1757, 2003, 2246, 2492, 2739, 2984, 3228, 3476, 3715, 3958, 4202, 4428, 4675, 4921, 5166, 5409, 5654, 5900, 6146, 6391, 6637, 6881, 7126, 7372, 7618, 7860, 8103, 8346, 8448, 8695, 8942, 9186, 9429, 9674, 9920, 10167, 10412, 10656, 10901, 11148, 11393, 11638, 11883, 12124, 12367, 12611, 12775, 13021, 13269, 13513, 13756, 14001, 14247, 14484, 14730, 14974, 15219, 15464, 15710, 15955, 16199, 16441, 16683, 16910]


In [7]:
dataframes = []

for i in range(len(indices) - 1):
    start_index = indices[i] - 500
    end_index = indices[i + 1]
    
    symbols_array = SET50_Index_Futures.iloc[end_index]['Symbol'][3:]
    
    df_name = f"S50{symbols_array}"
    df = SET50_Index_Futures.iloc[start_index:end_index + 1, :].reset_index(drop=True)
    
    df['Symbols_Array'] = symbols_array
    
    dataframes.append(df)

In [8]:
list_series = (['S50M06', 'S50M07', 'S50M08', 'S50M09', 'S50M10', 'S50M11',
       'S50M12', 'S50M13', 'S50M14', 'S50M15', 'S50M16', 'S50M17',
       'S50M18', 'S50M19', 'S50M20', 'S50M21', 'S50M22', 'S50M23',
       'S50H07', 'S50H08', 'S50H09', 'S50H10', 'S50H11', 'S50H12',
       'S50H13', 'S50H14', 'S50H15', 'S50H16', 'S50H17', 'S50H18',
       'S50H19', 'S50H20', 'S50H21', 'S50H22', 'S50H23', 'S50U06',
       'S50U07', 'S50U08', 'S50U09', 'S50U10', 'S50U11', 'S50U12',
       'S50U13', 'S50U14', 'S50U15', 'S50U16', 'S50U17', 'S50U18',
       'S50U19', 'S50U20', 'S50U21', 'S50U22', 'S50U23', 'S50Z06',
       'S50Z07', 'S50Z08', 'S50Z09', 'S50Z10', 'S50Z11', 'S50Z12',
       'S50Z13', 'S50Z14', 'S50Z15', 'S50Z16', 'S50Z17', 'S50Z18',
       'S50Z19', 'S50Z20', 'S50Z21', 'S50Z22', 'S50Z23'])

row_idx = ([   39,   285,   532,   777,  1020,  1264,  1510,  1757,  2003,
             2246,  2492,  2739,  2984,  3228,  3476,  3715,  3958,  4202,
             4428,  4675,  4921,  5166,  5409,  5654,  5900,  6146,  6391,
             6637,  6881,  7126,  7372,  7618,  7860,  8103,  8346,  8448,
             8695,  8942,  9186,  9429,  9674,  9920, 10167, 10412, 10656,
            10901, 11148, 11393, 11638, 11883, 12124, 12367, 12611, 12775,
            13021, 13269, 13513, 13756, 14001, 14247, 14484, 14730, 14974,
            15219, 15464, 15710, 15955, 16199, 16441, 16683, 16906])

series_dict = {}
window_size = 500

for series, idx in zip(list_series, row_idx):
    series_df = SET50_Index_Futures.iloc[(idx - window_size):(idx + 1), :].reset_index(drop=True)
    series_dict[series] = series_df

In [9]:
def load_options_data(Series, call_options_file_path, put_options_file_path):
    Futures_data = Series #pd.read_csv(futures_file_path)
    Futures = Futures_data[['Date', 'Symbol', 'SP', 'Open', 'High', 'Low', 'Close', 'Vol', 'OI']].drop_duplicates(subset=['Date']).sort_values(by='Date').reset_index(drop=True)
    Futures['Strike_Price'] = Futures['SP'].apply(round_to_nearest_25)
    Futures['Date'] = pd.to_datetime(Futures['Date'])
    Futures = Futures.sort_values(by='Date').reset_index(drop=True)

    Futures['log_return'] = np.log(Futures['Close'] / Futures['Close'].shift(1))
    Futures = Futures.iloc[1:, :].reset_index(drop=True)
    Futures['log_return'] = pd.to_numeric(Futures['log_return'], errors='coerce')
    Futures['Std'] = Futures['log_return'].rolling(window=248, min_periods=1).std()
    Futures['Annualized_Vol'] = Futures['Std'] * np.sqrt(248)

    Call_Options_data = pd.read_csv(call_options_file_path)
    Call_Options = Call_Options_data[['Date', 'Symbol', 'SP']]
    # Call_Options['Date'] = pd.to_datetime(Call_Options['Date'])
    Call_Options = Call_Options.sort_values(by='Date').reset_index(drop=True)

    Put_Options_data = pd.read_csv(put_options_file_path)
    Put_Options = Put_Options_data[['Date', 'Symbol', 'SP']]
    Put_Options = Put_Options.sort_values(by='Date').reset_index(drop=True)
    # Put_Options['Date'] = pd.to_datetime(Put_Options['Date'])    
    
    # select options first day trade
    first_day = pd.read_csv(call_options_file_path)['Date'].min()
    Futures['Date'] = pd.to_datetime(Futures['Date'])
    mask = Futures['Date'] >= pd.to_datetime(first_day)
    Futures = Futures[mask].reset_index(drop=True)
    Futures['Date'] = pd.to_datetime(Futures['Date'])   
    Futures = Futures.sort_values(by='Date').reset_index(drop=True)
    
    call_options_unique_symbols = Call_Options['Symbol'].unique()
    symbol_dataframes = {}
    
    for symbol in call_options_unique_symbols:
        symbol_df = pd.DataFrame({
            'Date': Call_Options[Call_Options['Symbol'] == symbol]['Date'],
            symbol: Call_Options[Call_Options['Symbol'] == symbol]['SP']
        })
        symbol_dataframes[symbol] = symbol_df
    
    merged_df = symbol_dataframes[call_options_unique_symbols[0]]
    for symbol in call_options_unique_symbols[1:]:
        merged_df = pd.merge(merged_df, symbol_dataframes[symbol], on='Date', how='outer')
    
    Call_Options_Clean = merged_df.drop_duplicates(subset=['Date']).sort_values(by='Date').reset_index(drop=True).fillna(0)
    Call_Options_Clean['Date'] = pd.to_datetime(Call_Options_Clean['Date']) 

    put_options_unique_symbols = Put_Options['Symbol'].unique()
    symbol_dataframes = {}
    
    for symbol in put_options_unique_symbols:
        symbol_df = pd.DataFrame({
            'Date': Put_Options[Put_Options['Symbol'] == symbol]['Date'],
            symbol: Put_Options[Put_Options['Symbol'] == symbol]['SP']
        })
        symbol_dataframes[symbol] = symbol_df
    
    merged_df = symbol_dataframes[put_options_unique_symbols[0]]
    for symbol in put_options_unique_symbols[1:]:
        merged_df = pd.merge(merged_df, symbol_dataframes[symbol], on='Date', how='outer')
    
    Put_Options_Clean = merged_df.drop_duplicates(subset=['Date']).sort_values(by='Date').reset_index(drop=True).fillna(0)
    Put_Options_Clean['Date'] = pd.to_datetime(Put_Options_Clean['Date']) 

    Data = Futures.merge(Call_Options_Clean, on='Date', how='outer').fillna(0)
    Data = Data.merge(Put_Options_Clean, on='Date', how='outer').fillna(0)
    Data['Date'] = pd.to_datetime(Data['Date'])
    Data = Data.sort_values(by='Date').reset_index(drop=True)
    Data['Strike_Price'] = Data['Strike_Price'].astype(int)
    
    return Data

def round_to_nearest_25(value):
    return 25 * round(value / 25)

def vlookup_call(row, file_prefix):
    strike_price = row['Strike_Price']
    column_name = f'{file_prefix}C{strike_price}'
    atm_value = row[column_name]

    return atm_value

def vlookup_put(row, file_prefix):
    strike_price = row['Strike_Price']
    column_name = f'{file_prefix}P{strike_price}'
    atm_value = row[column_name]

    return atm_value

In [10]:
list_series = (['S50M06', 'S50M07', 'S50M08', 'S50M09', 'S50M10', 'S50M11',
               'S50M12', 'S50M13', 'S50M14', 'S50M15', 'S50M16', 'S50M17',
               'S50M18', 'S50M19', 'S50M20', 'S50M21', 'S50M22', 'S50M23',
               'S50H07', 'S50H08', 'S50H09', 'S50H10', 'S50H11', 'S50H12',
               'S50H13', 'S50H14', 'S50H15', 'S50H16', 'S50H17', 'S50H18',
               'S50H19', 'S50H20', 'S50H21', 'S50H22', 'S50H23', 'S50U06',
               'S50U07', 'S50U08', 'S50U09', 'S50U10', 'S50U11', 'S50U12',
               'S50U13', 'S50U14', 'S50U15', 'S50U16', 'S50U17', 'S50U18',
               'S50U19', 'S50U20', 'S50U21', 'S50U22', 'S50U23', 'S50Z06',
               'S50Z07', 'S50Z08', 'S50Z09', 'S50Z10', 'S50Z11', 'S50Z12',
               'S50Z13', 'S50Z14', 'S50Z15', 'S50Z16', 'S50Z17', 'S50Z18',
               'S50Z19', 'S50Z20', 'S50Z21', 'S50Z22', 'S50Z23'])

def process_series_data(series_data, symbol_filter='S50Z14'):
    series_data = series_data.drop_duplicates(subset='Date').sort_values(by='Date', ascending=True).reset_index(drop=True)
    
    if 'Close' in series_data.columns and series_data['Close'].dtype == object:
        series_data['Close'] = pd.to_numeric(series_data['Close'].str.replace(',', ''), errors='coerce')
    
    series_data['SP'] = pd.to_numeric(series_data['SP'], errors='coerce')
    series_data['ATM'] = round(series_data['SP'] / 25) * 25

    series_data['log_return'] = np.log(series_data['Close'] / series_data['Close'].shift(1))
    series_data = series_data.iloc[1:, :].reset_index(drop=True)
    series_data['log_return'] = pd.to_numeric(series_data['log_return'], errors='coerce')
    series_data['Std'] = series_data['log_return'].rolling(window=248, min_periods=1).std()
    series_data['Annualized_Vol'] = series_data['Std'] * np.sqrt(248)
    
    series_data = series_data[series_data['Symbol'] == symbol_filter][['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'SP', 'ATM', 'Vol', 'OI', 'Annualized_Vol']].dropna().reset_index(drop=True)
    series_data['ATM'] = series_data['ATM'].astype(int)
    
    return series_data

result_dict = {}

for series_symbol in list_series:
    result_dict[series_symbol] = process_series_data(series_dict[series_symbol], symbol_filter=series_symbol)

In [11]:
S50Z07 = result_dict['S50Z07']
S50M08 = result_dict['S50M08']; S50H08 = result_dict['S50H08']; S50U08 = result_dict['S50U08']; S50Z08 = result_dict['S50Z08']
S50M09 = result_dict['S50M09']; S50H09 = result_dict['S50H09']; S50U09 = result_dict['S50U09']; S50Z09 = result_dict['S50Z09']
S50M10 = result_dict['S50M10']; S50H10 = result_dict['S50H10']; S50U10 = result_dict['S50U10']; S50Z10 = result_dict['S50Z10']

S50M11 = result_dict['S50M11']; S50H11 = result_dict['S50H11']; S50U11 = result_dict['S50U11']; S50Z11 = result_dict['S50Z11']
S50M12 = result_dict['S50M12']; S50H12 = result_dict['S50H12']; S50U12 = result_dict['S50U12']; S50Z12 = result_dict['S50Z12']
S50M13 = result_dict['S50M13']; S50H13 = result_dict['S50H13']; S50U13 = result_dict['S50U13']; S50Z13 = result_dict['S50Z13']
S50M14 = result_dict['S50M14']; S50H14 = result_dict['S50H14']; S50U14 = result_dict['S50U14']; S50Z14 = result_dict['S50Z14']
S50M15 = result_dict['S50M15']; S50H15 = result_dict['S50H15']; S50U15 = result_dict['S50U15']; S50Z15 = result_dict['S50Z15']

S50M16 = result_dict['S50M16']; S50H16 = result_dict['S50H16']; S50U16 = result_dict['S50U16']; S50Z16 = result_dict['S50Z16']
S50M17 = result_dict['S50M17']; S50H17 = result_dict['S50H17']; S50U17 = result_dict['S50U17']; S50Z17 = result_dict['S50Z17']
S50M18 = result_dict['S50M18']; S50H18 = result_dict['S50H18']; S50U18 = result_dict['S50U18']; S50Z18 = result_dict['S50Z18']
S50M19 = result_dict['S50M19']; S50H19 = result_dict['S50H19']; S50U19 = result_dict['S50U19']; S50Z19 = result_dict['S50Z19']
S50M20 = result_dict['S50M20']; S50H20 = result_dict['S50H20']; S50U20 = result_dict['S50U20']; S50Z20 = result_dict['S50Z20']

S50M21 = result_dict['S50M21']; S50H21 = result_dict['S50H21']; S50U21 = result_dict['S50U21']; S50Z21 = result_dict['S50Z21']
S50M22 = result_dict['S50M22']; S50H22 = result_dict['S50H22']; S50U22 = result_dict['S50U22']; S50Z22 = result_dict['S50Z22']
S50M23 = result_dict['S50M23']; S50H23 = result_dict['S50H23']; S50U23 = result_dict['S50U23']; S50Z23 = result_dict['S50Z23']

###### <b>2013</b>

In [ ]:
url = 'https://raw.githubusercontent.com/SamapanThongmee/SET_Index_Options_Premium_Analysis/main/S50U13_Call_options_data.csv'
SET50_Index_Futures = pd.read_csv(url)
SET50_Index_Futures['Date']= pd.to_datetime(SET50_Index_Futures['Date'])
SET50_Index_Futures

In [14]:
call_options_file_path = 'https://raw.githubusercontent.com/SamapanThongmee/SET_Index_Options_Premium_Analysis/main/S50U13_Call_options_data.csv'
call_options_file_path = pd.read_csv(call_options_file_path).iloc[:,1:]
call_options_file_path

,Date,Symbol,Open,High,Low,Close,SP,Vol,OI
0,2013-08-28,S50U13C825,0.0,0.0,0.0,0.0,58.10,0,0.0
1,2013-08-29,S50U13C825,64.9,64.9,64.9,64.9,64.90,1,1.0
2,2013-08-30,S50U13C825,56.1,69.9,56.1,69.9,65.90,2,2.0
3,2013-09-02,S50U13C825,0.0,0.0,0.0,0.0,81.50,0,2.0
4,2013-09-03,S50U13C825,79.9,79.9,79.9,79.9,77.90,1,2.0
...,...,...,...,...,...,...,...,...,...
1327,2013-09-23,S50U13C1150,0.0,0.0,0.0,0.0,0.10,0,11.0
1328,2013-09-24,S50U13C1150,0.0,0.0,0.0,0.0,0.10,0,11.0
1329,2013-09-25,S50U13C1150,0.0,0.0,0.0,0.0,0.10,0,11.0
1330,2013-09-26,S50U13C1150,0.0,0.0,0.0,0.0,0.10,0,11.0


In [15]:
# Export data
futures_file_path = S50U13
call_options_file_path = 'https://raw.githubusercontent.com/SamapanThongmee/SET_Index_Options_Premium_Analysis/main/S50U13_Call_options_data.csv'
call_options_file_path = pd.read_csv(call_options_file_path).iloc[:,1:]
put_options_file_path = 'https://raw.githubusercontent.com/SamapanThongmee/SET_Index_Options_Premium_Analysis/main/S50U13_Put_options_data.csv'
put_options_file_path = pd.read_csv(put_options_file_path).iloc[:,1:]

data = load_options_data(futures_file_path, call_options_file_path, put_options_file_path)
data['Call_ATM'] = data.apply(vlookup_call, axis=1, args=('S50U13',))
data['Put_ATM'] = data.apply(vlookup_put, axis=1, args=('S50U13',))

data = data.rename(columns={'SP': 'Settlement Price', 'Strike_Price': 'Strike Price', 'Call_ATM': 'Call ATM', 'Put_ATM': 'Put ATM', 'Annualized_Vol':'Annualized Vol'})
data['Sum ATM Premium'] = data['Strike Price']/(data['Call ATM'] + data['Put ATM'])
data = data[['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Settlement Price', 'Strike Price', 'Annualized Vol', 'Call ATM', 'Put ATM', 'Sum ATM Premium']]
data.to_csv('Options_Premium_S50M17.csv', index=False, index_label=None)
S50U13 = data
S50U13

TypeError: argument of type 'method' is not iterable